# Word Predictor

Goal of the program is to write a program that completes a sentence give a word.

In [13]:
import nltk
import re
import random
from collections import Counter

In [14]:
f = open('./big.txt', 'r').read().replace('\n', '')

In [15]:
sentences = re.split('[!?.]+(?=$|\s)',f)

In [16]:
print('we have ',len(sentences),' sentences')

('we have ', 35102, ' sentences')


In [24]:
bigram_count = Counter()
unigram_count = Counter()

In [25]:
# use 1/10th of the data to train the model for now
for sentence in sentences[:len(sentences)/10]:
    bigram_count.update(nltk.bigrams( (sentence +' [end]' ).split(' ')))
    unigram_count.update((sentence+' [end]').split(' '))

In [27]:
unigram_count['[end]']

3510

Very simple here. We have a sentece (w1,w2,w3,....) we want to predict the next word in the sentence:
$$\text{argmax}_{w2} P(w2 | w1) = \text{argmax}_{w2}N(w2,w1)$$ where N is just a count of the bigram $(w2,w1)$

In [28]:
def generator(first_word):
    words_so_far = [ first_word ]
    score = [0]
    latest_word = first_word
    counter_function = lambda word: bigram_count[(latest_word, word)]/float(words_so_far.count(word) + random.randint(1,4)) 
    while latest_word != '[end]' and len(words_so_far) < 50:
        generate_word = max(unigram_count.keys()[1:],key=counter_function)
        score.append(counter_function(generate_word))
        words_so_far.append(generate_word)
        latest_word = generate_word
    return words_so_far, score

Generating a sentence with the first word as "why" based on 1/10th of the training data.

In [29]:
generator('why')

(['why', 'I', 'am', 'a', 'few', 'minutes', 'to', 'the', 'other', '[end]'],
 [0, 1.6666666666666667, 49.5, 4.0, 11.75, 6.5, 0.75, 253.0, 54.0, 3.0])

In [30]:
# add more training data! almost all of it..
for sentence in sentences[len(sentences)/10:9*len(sentences)/10]:
    bigram_count.update(nltk.bigrams( (sentence +' [end]' ).split(' ')))
    unigram_count.update((sentence+' [end]').split(' '))

In [31]:
generator('why')

(['why',
  'should',
  'be',
  'a',
  'man',
  'who',
  'was',
  'not',
  'only',
  'by',
  'the',
  'French',
  '[end]'],
 [0,
  12.0,
  209.5,
  80.0,
  107.5,
  34.0,
  209.0,
  324.0,
  28.0,
  30.0,
  1372.0,
  63.75,
  8.0])

As we can see the sentence makes much more sense...

In [32]:
for sentence in sentences[9*len(sentences)/10:]:
    bigram_count.update(nltk.bigrams( (sentence +' [end]' ).split(' ')))
    unigram_count.update((sentence+' [end]').split(' '))

In [34]:
generator('why')

(['why',
  'the',
  'first',
  'and',
  'the',
  'French',
  'had',
  'been',
  'in',
  'the',
  'United',
  'States',
  'shall',
  'be',
  'a',
  'large',
  'proportion',
  'of',
  'the',
  'same',
  'time',
  'to',
  'be',
  'a',
  'very',
  'glad',
  'to',
  'the',
  'same',
  'time',
  '[end]'],
 [0,
  2.75,
  107.25,
  10.5,
  592.0,
  389.0,
  29.0,
  879.0,
  31.5,
  944.8,
  107.33333333333333,
  69.33333333333333,
  4.333333333333333,
  51.666666666666664,
  91.5,
  37.5,
  3.0,
  9.333333333333334,
  2018.2,
  234.66666666666666,
  22.0,
  44.0,
  439.6666666666667,
  91.5,
  165.0,
  6.0,
  7.4,
  697.8,
  352.0,
  44.0,
  29.0])

Sentence makes more sense... However, we still lose the context as the number of words grows. To fix this we can specify a context or topic $T$, or we could even use a trigram model! 